<a href="https://colab.research.google.com/github/Jagadeep8686/MTP/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import networkx as nx
import torch.nn.functional as F



In [15]:
g = nx.karate_club_graph()
A = nx.to_numpy_matrix(g)
X = np.eye(g.number_of_nodes())

In [14]:
A_t = A + np.eye(g.number_of_nodes()) # add self-connections

D_t = np.zeros_like(A_t)
np.fill_diagonal(D_t, np.asarray(A_t.sum(axis=1)).flatten())


D_t_invroot = np.linalg.inv(np.power(D_t,0.5))

A_hat = D_t_invroot @ A_t @ D_t_invroot

In [16]:
def glorot_init(nin, nout):
    sd = np.sqrt(2.0 / (nin + nout))
    return np.random.uniform(-sd, sd, size=(nin, nout))

In [20]:

class GradDes():
    def __init__(self, lr, wd):
        self.lr = lr
        self.wd = wd
        self._y_pred = None
        self._y_true = None
        self._out = None
        self.bs = None
        self.train_nodes = None
        
    def __call__(self, y_pred, y_true, train_nodes=None):
        self.y_pred = y_pred
        self.y_true = y_true
        
        if train_nodes is None:
            self.train_nodes = np.arange(y_pred.shape[0])
        else:
            self.train_nodes = train_nodes
            
        self.bs = self.train_nodes.shape[0]
        
    @property
    def out(self):
        return self._out
    
    @out.setter
    def out(self, y):
        self._out = y

In [26]:
class GCNLayer():
    def __init__(self, n_inputs, n_outputs, activation=None):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = glorot_init(self.n_outputs, self.n_inputs)
        self.activation = activation
        
        
    def forward(self, A, X, W=None):
        self._A = A
        self._X = (A @ X).T  
        
        if W is None:
            W = self.W
        
        H = W @ self._X 
        if self.activation is not None:
            H = self.activation(H)
        self._H = H  
        return self._H.T

    def backward(self, GradDes, update=True):
        dtanh = 1 - np.asarray(self._H.T)**2  
        d2 = np.multiply(GradDes.out, dtanh)   
        
        self.grad = self._A @ d2 @ self.W       
        GradDes.out = self.grad
        
        dW = np.asarray(d2.T @ self._X.T) / GradDes.bs   
        dW_wd = self.W * GradDes.wd / GradDes.bs  
        
        if update:
            self.W -= (dW + dW_wd) * GradDes.lr 
        
        return dW + dW_wd

In [27]:
class GCN():
    def __init__(self,inputs, outputs,activation):
        super().__init__()
        self.conv1 = GCNLayer(inputs, 64,activation)
        self.conv2 = GCNLayer(64, 32,activation)
        self.conv3 = GCNLayer(32, outputs,activation)

    def forward(self,X,A):

        X = self.conv1(X, A)
        X = F.dropout(X, training=self.training)
        X = self.conv2(X,A)
        X = F.dropout(X, training=self.training)
        X = self.conv3(X, A)
        
        return F.log_softmax(X, dim=1)